# LLM LoRA fine-tuning

⚠️ **IMPORTANT:** Before running this notebook, copy it into your user directory at `user/<your-username>/`.  
This ensures that any output or intermediate checkpoints are stored in your personal workspace.

Once copied, follow the environment setup instructions in the [README.md](./README.md), and connect this notebook to the `.venv` environment you created during setup.

In [1]:
from finetune import (
    CONFIGS,
    get_device,
    load_and_prepare_dataset,
    tokenize_dataset,
    create_lora_model,
    run_experiment,
)

/storage/home/hcoda1/6/ssureja3/dsgt-arc/fall-2025-interest-group-projects/project/01-llm-lora/.venv/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/storage/home/hcoda1/6/ssureja3/dsgt-arc/fall-2025-interest-group-projects/project/01-llm-lora/.venv/lib/python3.14/site-packages/multiprocess/connection.py:335: SyntaxWarning: 'return' in a 'finally' block
  return f
/storage/home/hcoda1/6/ssureja3/dsgt-arc/fall-2025-interest-group-projects/project/01-llm-lora/.venv/lib/python3.14/site-packages/multiprocess/connection.py:337: SyntaxWarning: 'return' in a 'finally' block
  return self._get_more_data(ov, maxsize)


In [2]:
# choose config
config = CONFIGS["quick"]  # or "full"
device = get_device()
print(f"Running LoRA fine-tuning on {config['dataset_name']} ({config['model_name']})")

Running LoRA fine-tuning on rotten_tomatoes (roberta-base)


In [3]:
# load and tokenize data
dataset, text_field = load_and_prepare_dataset(config["dataset_name"])
tokenized_dataset, tokenizer = tokenize_dataset(
    dataset, text_field, config["model_name"], config["max_length"]
)

Tokenizing: 100%|██████████| 1066/1066 [00:00<00:00, 13215.01 examples/s]


In [4]:
# create and train LoRA model
lora_alpha = config["lora_rank"] * config["lora_alpha_ratio"]
model = create_lora_model(
    config["model_name"],
    config["num_labels"],
    rank=config["lora_rank"],
    alpha=lora_alpha,
)

results, model = run_experiment(
    model=model,
    device=device,
    tokenized_dataset=tokenized_dataset,
    config=config,
    experiment_name="LoRA Fine-Tuning",
    learning_rate=config["learning_rate_lora"],
    is_full_finetuning=False,
)

print("\n=== LoRA Fine-Tuning Results ===")
for k, v in results.items():
    print(f"{k:20s}: {v}")

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['heads.default.3.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
There are adapters available but none are activated for the forward pass.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.339300,0.324705,0.868668
2,0.287900,0.312493,0.873358
3,0.260100,0.320551,0.877111



=== LoRA Fine-Tuning Results ===
experiment_name     : LoRA Fine-Tuning
training_time       : 101.24919748306274
train_loss          : 0.3402994330158543
eval_loss           : 0.3205505609512329
eval_accuracy       : 0.8771106941838649
total_params        : 126248795
trainable_params    : 1603163
trainable_percentage: 1.269844199305031
